# Qwen2 源代码解析
Qwen2的源代码取自[Hugging Face Transformers](https://github.com/huggingface/transformers)项目中路径为`src/transformers/models/qwen2/modeling_qwen2.py`的文件，这里仅展示部分代码的分析；其技术报告可参考arXiv论文[QWEN2 TECHNICAL REPORT](https://arxiv.org/abs/2407.10671)，该报告也随上述源代码文件一同附置，可供查阅。

> 代码来自源文件，仅供参考和学习使用，实际并不能直接运行

## Qwen2 模型配置
该配置类用于存储 [`Qwen2Model`] 模型的核心配置信息，可根据指定参数实例化通义千问2（Qwen2）模型并定义其整体架构。使用默认参数实例化时，将生成与 Qwen2-7B-beta 模型（模型地址：[Qwen/Qwen2-7B-beta](https://huggingface.co/Qwen/Qwen2-7B-beta)）近乎一致的配置。

本配置类继承自 [`PreTrainedConfig`] 基类，可用于控制模型输出行为，更多细节请参考 [`PreTrainedConfig`] 官方文档。

### 参数说明
| 参数名 | 类型 | 可选/必填 | 默认值 | 说明 |
|--------|------|-----------|--------|------|
| vocab_size | `int` | 可选 | 151936 | Qwen2 模型的词汇表大小，定义调用 [`Qwen2Model`] 时 `inputs_ids` 可表征的不同 token（标记）数量。 |
| hidden_size | `int` | 可选 | 4096 | 模型隐藏层表征的维度。 |
| intermediate_size | `int` | 可选 | 22016 | 多层感知机（MLP）层的表征维度。 |
| num_hidden_layers | `int` | 可选 | 32 | Transformer 解码器中隐藏层的数量。 |
| num_attention_heads | `int` | 可选 | 32 | Transformer 解码器中每个注意力层的注意力头数量。 |
| num_key_value_heads | `int` | 可选 | 32 | 分组查询注意力（Grouped Query Attention，GQA）的键值头数量：<br>- 当 `num_key_value_heads = num_attention_heads` 时，使用多头注意力（MHA）；<br>- 当 `num_key_value_heads = 1` 时，使用多查询注意力（MQA）；<br>- 其余情况使用 GQA。<br>将 MHA 权重转换为 GQA 权重时，需对每个分组内的原始注意力头做均值池化生成键头和值头。更多细节参考论文：[《GQA: Training Generalized Multi-Query Transformer Models from Multi-Head Checkpoints》](https://huggingface.co/papers/2305.13245)。 |
| hidden_act | `str` / `function` | 可选 | `"silu"` | 解码器中使用的非线性激活函数（支持函数对象或字符串名称）。 |
| max_position_embeddings | `int` | 可选 | 32768 | 模型可处理的最大序列长度。 |
| initializer_range | `float` | 可选 | 0.02 | 权重矩阵初始化时，截断正态分布初始化器（truncated_normal_initializer）的标准差。 |
| rms_norm_eps | `float` | 可选 | 1e-06 | 均方根归一化（RMSNorm）层的极小值（用于防止分母为 0）。 |
| use_cache | `bool` | 可选 | `True` | 模型是否返回最后一轮的键/值注意力缓存（仅当 `config.is_decoder = True` 时生效，并非所有模型均使用该参数）。 |
| tie_word_embeddings | `bool` | 可选 | `False` | 是否绑定模型的输入词嵌入与输出词嵌入（共享权重）。 |
| rope_parameters | `RopeParameters` | 可选 | - | 旋转位置编码（RoPE）的配置参数字典，必须包含 `rope_theta`；若需适配更长的 `max_position_embeddings`，可选择性配置缩放相关参数。 |
| use_sliding_window | `bool` | 可选 | `False` | 是否启用滑动窗口注意力（Sliding Window Attention，SWA）机制。 |
| sliding_window | `int` | 可选 | 4096 | 滑动窗口注意力的窗口大小。 |
| max_window_layers | `int` | 可选 | 28 | 使用全注意力机制的层数：前 `max_window_layers` 层使用全注意力，后续层使用滑动窗口注意力。 |
| layer_types | `list` | 可选 | - | 每层对应的注意力模式配置列表。 |
| attention_dropout | `float` | 可选 | 0.0 | 注意力权重的 Dropout 丢弃比例。 |
| pad_token_id | `int` | 可选 | - | 填充标记（Padding Token）的 ID。 |
| bos_token_id | `int` | 可选 | - | 序列起始标记（Beginning of Stream）的 ID。 |
| eos_token_id | `int` | 可选 | - | 序列结束标记（End of Stream）的 ID。 |


In [ ]:
class Qwen2Config(PreTrainedConfig):
    model_type = "qwen2"  
    keys_to_ignore_at_inference = ["past_key_values"]  # 在推理过程中忽略 `past_key_values` 键，因为它们不影响模型的输出结果

    # 基础模型 Qwen2 的默认张量并行方案
    base_model_tp_plan = {
        "layers.*.self_attn.q_proj": "colwise",  # 采用列并行方式分割查询投影矩阵
        "layers.*.self_attn.k_proj": "colwise",
        "layers.*.self_attn.v_proj": "colwise",
        "layers.*.self_attn.o_proj": "rowwise",  # 采用行并行方式分割输出投影矩阵
        "layers.*.mlp.gate_proj": "colwise",
        "layers.*.mlp.up_proj": "colwise",
        "layers.*.mlp.down_proj": "rowwise",
    }
    base_model_pp_plan = {
        "embed_tokens": (["input_ids"], ["inputs_embeds"]),  # 输入 ID 映射到嵌入向量的层，输入为 `input_ids`，输出为 `inputs_embeds`
        "layers": (["hidden_states", "attention_mask"], ["hidden_states"]),  # 每层的输入为隐藏状态和注意力掩码，输出为新的隐藏状态
        "norm": (["hidden_states"], ["hidden_states"]),  # 最后一个层归一化层，输入为隐藏状态，输出为新的隐藏状态
    }
    
    # 初始化配置类的参数，提供默认值以生成与 Qwen2-7B-beta 模型相近的配置
    def __init__(
        self,
        vocab_size: int | None = 151936,  # 词汇表大小，决定了模型能够处理的不同标记数量，通常与预训练时使用的 tokenizer 相关
        hidden_size: int | None = 4096,  # 隐藏层表征的维度
        intermediate_size: int | None = 22016,  # 前馈网络中间层的维度，通常是隐藏层维度的 4-8 倍
        num_hidden_layers: int | None = 32,  # 模型的层数，即 Transformer 块的数量
        num_attention_heads: int | None = 32,  # 注意力头的数量，决定了每层中多头注意力机制的并行度
        num_key_value_heads: int | None = 32,  # 键值头的数量，通常与 `num_attention_heads` 保持一致，但在某些模型中可能不同
        hidden_act: str | None = "silu",  # 隐藏层的激活函数，常用的有 "relu"、"gelu"、"silu"（也称为 swish）等
        max_position_embeddings: int | None = 32768,  # 模型能够处理的最大序列长度，决定了位置编码的范围
        initializer_range: float | None = 0.02,  # 权重初始化的标准差，影响模型训练的稳定性和收敛速度
        rms_norm_eps: int | None = 1e-6,  # RMSNorm 的 epsilon 值，用于防止分母为 0，确保数值稳定性
        use_cache: bool | None = True,  # 是否在推理过程中使用缓存机制，以加速生成过程
        tie_word_embeddings: bool | None = False,  # 是否共享输入和输出的词嵌入权重，通常在语言模型中使用以减少参数数量
        rope_parameters: RopeParameters | dict[str, RopeParameters] | None = None,  # 旋转位置编码（RoPE）的参数，可以是单个 `RopeParameters` 实例或一个字典，支持不同层使用不同的 RoPE 参数
        use_sliding_window: bool | None = False,  # 是否使用滑动窗口注意力机制，以处理长序列输入，减少计算复杂度
        sliding_window: int | None = 4096,  # 滑动窗口的大小，决定了每层中使用滑动窗口注意力机制的范围
        max_window_layers: int | None = 28,  # 使用滑动窗口注意力机制的最大层数，超过该层数的层将使用全局注意力机制
        layer_types: list[str] | None = None,  # 每层的注意力类型列表，支持 "full_attention"（全局注意力）和 "sliding_attention"（滑动窗口注意力），如果未指定，将根据 `use_sliding_window` 和 `max_window_layers` 自动生成
        attention_dropout: float | None = 0.0,  # 注意力机制中的 dropout 概率，用于防止过拟合
        pad_token_id: int | None = None,  # 填充标记的 ID，用于在批处理时对齐输入序列
        bos_token_id: int | None = None,  # 序列开始标记的 ID，通常用于生成任务的输入序列开头
        eos_token_id: int | None = None,  # 序列结束标记的 ID，通常用于生成任务的输入序列结尾
        **kwargs,
    ):
        self.vocab_size = vocab_size
        self.max_position_embeddings = max_position_embeddings
        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.use_sliding_window = use_sliding_window
        self.sliding_window = sliding_window if self.use_sliding_window else None
        self.max_window_layers = max_window_layers

        if num_key_value_heads is None:
            num_key_value_heads = num_attention_heads

        self.num_key_value_heads = num_key_value_heads
        self.hidden_act = hidden_act
        self.initializer_range = initializer_range
        self.rms_norm_eps = rms_norm_eps
        self.use_cache = use_cache
        self.attention_dropout = attention_dropout

        self.layer_types = layer_types
        if self.layer_types is None:
            self.layer_types = [
                "sliding_attention"
                if self.sliding_window is not None and i >= self.max_window_layers  # 超过 `max_window_layers` 的层使用滑动窗口注意力机制
                else "full_attention"
                for i in range(self.num_hidden_layers)  # 根据层数自动生成注意力类型列表，前 `max_window_layers` 层使用全局注意力机制，剩余层使用滑动窗口注意力机制
            ]
        layer_type_validation(self.layer_types, self.num_hidden_layers)  # 验证 `layer_types` 列表的长度和内容是否符合要求，确保每层都有有效的注意力类型

        self.pad_token_id = pad_token_id
        self.bos_token_id = bos_token_id
        self.eos_token_id = eos_token_id
        self.tie_word_embeddings = tie_word_embeddings
        self.rope_parameters = rope_parameters

        super().__init__(**kwargs)

# 定义模块的公开接口，指定在使用 `from module import *` 时导入的对象列表，这里仅包含 `Qwen2Config` 类
__all__ = ["Qwen2Config"]  

## Qwen2 Tokenization 相关类

In [ ]:
from tokenizers import AddedToken, Regex, Tokenizer, decoders, normalizers, pre_tokenizers
from tokenizers.models import BPE

VOCAB_FILES_NAMES = {
    "vocab_file": "vocab.json",  # 词汇表文件，包含标记到 ID 的映射关系，通常是一个 JSON 文件
    "merges_file": "merges.txt",  # BPE 合并规则文件，定义了如何将基本标记合并成更长的标记，通常是一个文本文件，每行包含一个合并规则
    "tokenizer_file": "tokenizer.json",  # 整合了词汇表和合并规则的 tokenizer 文件，包含了完整的 tokenizer 配置和数据，通常是一个 JSON 文件
}

MAX_MODEL_INPUT_SIZES = {"qwen/qwen-tokenizer": 32768}  # 模型输入的最大长度，决定了 tokenizer 能够处理的最长序列长度，通常与模型的 `max_position_embeddings` 参数一致

# 定义预处理文本的正则表达式，用于在 tokenizer 中进行预分词操作，支持多种语言和标点符号的处理
PRETOKENIZE_REGEX = r"""(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+"""

# 定义 Qwen2Tokenizer 类，继承自 TokenizersBackend，负责将文本转换为模型可接受的输入格式，包括标记化、编码和解码等功能
class Qwen2Tokenizer(TokenizersBackend):
    vocab_files_names = VOCAB_FILES_NAMES
    model_input_names = ["input_ids", "attention_mask"]  # 模型输入的名称列表，通常包括 `input_ids`（标记 ID）和 `attention_mask`（注意力掩码），用于指示哪些标记是实际输入，哪些是填充
    model = BPE  # 使用 BPE（Byte Pair Encoding）作为 tokenizer 的基础模型，适用于处理自然语言文本，能够有效地处理未登录词和长尾词汇

    def __init__(
        self,
        vocab: str | dict[str, int] | None = None,
        merges: str | list[str] | None = None,
        unk_token: str = "<|endoftext|>",  # 未登录标记的字符串，通常用于表示 tokenizer 无法识别的标记，这里使用 `<|endoftext|>` 作为默认值
        bos_token=None,  # 序列开始标记的字符串，通常用于生成任务的输入序列开头，这里默认值为 None，表示不使用特殊的开始标记
        eos_token: str = "<|endoftext|>",  # 序列结束标记的字符串，通常用于生成任务的输入序列结尾，这里使用 `<|endoftext|>` 作为默认值
        pad_token: str = "<|endoftext|>",  # 填充标记的字符串，用于在批处理时对齐输入序列，这里使用 `<|endoftext|>` 作为默认值，表示填充标记与未登录标记相同
        add_prefix_space=None,  # 是否在输入文本前添加一个空格，通常用于处理某些语言（如英语）中的单词边界问题，这里默认值为 None，表示不添加前缀空格
        **kwargs,  # 其他可选参数，允许用户在初始化 tokenizer 时传入额外的配置选项，这些选项将被传递给父类的构造函数，以支持更多的自定义功能
    ):
        self.add_prefix_space = add_prefix_space if add_prefix_space is not None else False
        self._vocab = (
            vocab
            if vocab is not None
            else {
                "<|endoftext|>": 0,
            }
        )
        self._merges = merges or []  # BPE 合并规则列表，如果未提供，则默认为一个空列表，表示没有任何合并规则
        self._tokenizer = Tokenizer(
            BPE(
                vocab=self._vocab,
                merges=self._merges,  # BPE 合并规则列表，定义了如何将基本标记合并成更长的标记，通常是一个文本文件，每行包含一个合并规则
                dropout=None,
                unk_token=None,  # 未登录标记的字符串，通常用于表示 tokenizer 无法识别的标记，这里设置为 None，表示不使用特殊的未登录标记
                continuing_subword_prefix="",  # 连续子词前缀，通常用于表示一个标记是另一个标记的子词，这里设置为空字符串，表示不使用连续子词前缀
                end_of_word_suffix="",  # 单词结尾后缀，通常用于表示一个标记是一个单词的结尾，这里设置为空字符串，表示不使用单词结尾后缀
                fuse_unk=False,  # 是否将未登录标记与其他标记融合，通常用于处理未登录词的情况，这里设置为 False，表示不进行融合
                byte_fallback=False,  # 是否启用字节回退机制，当 tokenizer 无法识别某个标记时，是否将其分解为字节级标记进行处理，这里设置为 False，表示不启用字节回退机制
            )
        )
        self._tokenizer.decoder = decoders.ByteLevel()  # 使用 ByteLevel 解码器，将标记 ID 转换回原始文本，适用于处理字节级标记
        self._tokenizer.normalizer = normalizers.NFC()  # 使用 NFC（Normalization Form C）规范化器，对输入文本进行 Unicode 规范化，确保文本的一致性和兼容性
        self._tokenizer.pre_tokenizer = pre_tokenizers.Sequence(
            [
                pre_tokenizers.Split(
                    Regex(PRETOKENIZE_REGEX),  # 使用预定义的正则表达式进行预分词操作，将输入文本分割成更小的单元，适用于处理多种语言和标点符号
                    behavior="isolated",  # 分割行为，"isolated" 表示将匹配的文本作为独立的标记处理，而不是与其他文本合并
                    invert=False,  # 是否反转分割逻辑，False 表示按照正则表达式定义的方式进行分割，True 表示将正则表达式匹配的文本作为分割点，而不是作为独立的标记处理
                ),
                pre_tokenizers.ByteLevel(
                    add_prefix_space=self.add_prefix_space,  # 是否在输入文本前添加一个空格，通常用于处理某些语言（如英语）中的单词边界问题，这里使用 `self.add_prefix_space` 的值来决定是否添加前缀空格
                    use_regex=False,  # 是否使用正则表达式进行分割，False 表示按照字节级别进行分割，适用于处理字节级标记
                ),
            ]
        )

        super().__init__(
            unk_token=unk_token,
            bos_token=bos_token,
            eos_token=eos_token,
            pad_token=pad_token,
            add_prefix_space=add_prefix_space,
            **kwargs,
        )

        # 将所有特殊标记添加到 tokenizer 的词汇表中，确保它们能够被正确地识别和处理
        self.add_tokens([AddedToken(token, special=True) for token in self.all_special_tokens]) 

# 定义模块的公开接口，指定在使用 `from module import *` 时导入的对象列表，这里仅包含 `Qwen2Tokenizer` 类
__all__ = ["Qwen2Tokenizer"]

## Qwen2 核心代码部分

In [ ]:
from collections.abc import Callable
from typing import Optional
import torch
from torch import nn
from .configuration_qwen2 import Qwen2Config

In [ ]:
# 定义 Qwen2MLP 类，继承自 nn.Module，负责实现 Qwen2 模型中的前馈网络部分，包括门控机制和线性变换等功能
class Qwen2MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.hidden_size = config.hidden_size
        self.intermediate_size = config.intermediate_size
        # 门控投影层，将输入的隐藏状态映射到前馈网络中间层的维度，通常用于实现门控机制
        self.gate_proj = nn.Linear(self.hidden_size, self.intermediate_size, bias=False) 
        # 前馈网络的上投影层，将输入的隐藏状态映射到前馈网络中间层的维度，通常用于实现前馈网络的线性变换
        self.up_proj = nn.Linear(self.hidden_size, self.intermediate_size, bias=False)
        # 前馈网络的下投影层，将前馈网络中间层的输出映射回隐藏层的维度，通常用于实现前馈网络的线性变换
        self.down_proj = nn.Linear(self.intermediate_size, self.hidden_size, bias=False)
        # 激活函数，根据配置中的 `hidden_act` 参数选择相应的激活函数，常用的有 "relu"、"gelu"、"silu"（也称为 swish）等
        self.act_fn = ACT2FN[config.hidden_act]

    def forward(self, x):
        # 计算门控机制的输出
        # 最常用的是SwiGLU门控机制，即将门控投影层的输出经过激活函数处理后与上投影层的输出进行逐元素相乘，最后通过下投影层映射回隐藏层的维度
        down_proj = self.down_proj(self.act_fn(self.gate_proj(x)) * self.up_proj(x))
        return down_proj

### Rotary Position Embedding, RoPE

旋转位置编码是一种能将相对位置信息集成到 self-attention 中, 进而提升 transformer 架构性能的位置编码方式, 和绝对位置编码相比, RoPE 具有很好的外推性, 是目前的主流位置编码方式.

外推性的解释, 通俗来说就是训练的时候限制了 512 的上下文长度，那么推理时如果面对超过该长度的文本，LLM 可能无法正确处理.

- **绝对位置编码**

绝对位置编码是早期 Transformer 架构采用的绝对位置编码方案，及那个每个位置映射为固定的向量表示.

$$f_{t:t\in\{q,k,v\}}(\boldsymbol{x}_i,i)=\boldsymbol{W}_{t:t\in\{q,k,v\}}(\boldsymbol{x}_i+\boldsymbol{p}_i)$$

其中编码向量 $p_i$ 的计算使用如下公式：

$$\boldsymbol{p}_{i,2t}=\sin\left(k/10000^{2t/d}\right), \boldsymbol{p}_{i,2t+1}=\cos\left(k/10000^{2t/d}\right)$$

正如其名，绝对位置编码只考虑了输入序列中的绝对位置关系，对于 token 之间的相对信息则没有纳入考虑.

- **旋转位置编码**

假定 query 和 key 的内积操作可以被函数 g 表示，该函数 g 的输入是词嵌入向量 $x_m, x_n$ 和它们之间的相对位置 $m-n$:

$$<f_q(x_m ,m), f_k(x_n, n)>=g(x_m, x_n, m - n)$$

旋转位置编码就是找到一个使上式成立的位置编码方式. 

出于认识的目的，我们省略复杂的数学推导，直接看 RoPE 的的结论：

存在这样一个正交矩阵：

$$\boldsymbol{R}_{\Theta,m}^d=\underbrace{\begin{pmatrix}\cos m\theta_0&-\sin m\theta_0&0&0&\cdots&0&0\\\sin m\theta_0&\cos m\theta_0&0&0&\cdots&0&0\\0&0&\cos m\theta_1&-\sin m\theta_1&\cdots&0&0\\0&0&\sin m\theta_1&\cos m\theta_1&\cdots&0&0\\\vdots&\vdots&\vdots&\vdots&\ddots&\vdots&\vdots\\0&0&0&0&\cdots&\cos m\theta_{d/2-1}&-\sin m\theta_{d/2-1}\\
0&0&0&0&\cdots&\sin m\theta_{d/2-1}&\cos m\theta_{d/2-1}\end{pmatrix}}_{\boldsymbol{W}_m}$$

其中，$\Theta=\left\{\theta_i=10000^{-2(i-1)/d},i\in[1,2,\ldots,d/2]\right\}$

我们可以将 query 和 key 的内积操作转换为与原始向量 $x$ 相关的以下等价形式：

$$
\boldsymbol{q}_m^\mathbf{T}\boldsymbol{k}_n=\left(\boldsymbol{R}_{\Theta,m}^d\boldsymbol{W}_q\boldsymbol{x}_m\right)^\mathbf{T}\left(\boldsymbol{R}_{\Theta,n}^d\boldsymbol{W}_k\boldsymbol{x}_n\right)=\boldsymbol{x}_m^\mathbf{T}\boldsymbol{W}_q\boldsymbol{R}_{\Theta,n-m}^d\boldsymbol{W}_k\boldsymbol{x}_n
$$

其中， $\boldsymbol{R}_{\Theta,n-m}^d=\left(\boldsymbol{R}_{\Theta,m}^d\right)^\mathbf{T}\boldsymbol{R}_{\Theta,n}^d$.

由于 $\boldsymbol{R}_{\Theta,m}^d$ 的稀疏性，直接使用矩阵乘法会浪费算力，因此代码中采用下述方式实现：

$$\boldsymbol{R}_{\Theta,m}^{d}\boldsymbol{x}=\begin{pmatrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\\vdots\\x_{d-2}\\x_{d-1}\end{pmatrix}\otimes\begin{pmatrix}\cos m\theta_{0}\\\cos m\theta_{0}\\\cos m\theta_{1}\\\cos m\theta_{1}\\\vdots\\\cos m\theta_{d/2-1}\\\cos m\theta_{d/2-1}\end{pmatrix}+\begin{pmatrix}-x_{1}\\x_{0}\\-x_{3}\\x_{2}\\\vdots\\-x_{d-1}\\x_{d-2}\end{pmatrix}\otimes\begin{pmatrix}\sin m\theta_{0}\\\sin m\theta_{0}\\\sin m\theta_{1}\\\sin m\theta_{1}\\\vdots\\\sin m\theta_{d/2-1}\\\sin m\theta_{d/2-1}\end{pmatrix}
$$

In [ ]:
# 定义 Qwen2RotaryEmbedding 类，继承自 nn.Module，负责实现 Qwen2 模型中的旋转位置编码（RoPE）功能，包括计算 RoPE 的逆频率和应用 RoPE 的前向传播逻辑
class Qwen2RotaryEmbedding(nn.Module):
    inv_freq: torch.Tensor  # 定义一个属性 `inv_freq`，用于存储 RoPE 的逆频率张量，通常在前向传播过程中使用，以计算位置编码的旋转角度

    def __init__(self, config: Qwen2Config, device=None):
        super().__init__()
        self.max_seq_len_cached = config.max_position_embeddings
        self.original_max_seq_len = config.max_position_embeddings  # 存储原始的最大序列长度，以便在需要时进行动态更新

        self.config = config

        # 从配置中获取 RoPE 的类型，决定了使用哪种方式来计算 RoPE 的参数，支持 "default"（原始 RoPE）和其他自定义的 RoPE 类型
        self.rope_type = self.config.rope_parameters["rope_type"]  
        # 根据 RoPE 的类型选择相应的初始化函数，默认使用 `compute_default_rope_parameters` 方法来计算 RoPE 的参数
        # 如果配置中指定了其他 RoPE 类型，则从 `ROPE_INIT_FUNCTIONS` 字典中获取对应的函数
        rope_init_fn: Callable = self.compute_default_rope_parameters
        # # YaRN 可以在此处被调用，以根据配置中的 RoPE 类型动态生成计算 RoPE 参数的函数，支持用户自定义的 RoPE 实现逻辑
        if self.rope_type != "default":
            rope_init_fn = ROPE_INIT_FUNCTIONS[self.rope_type]
        # 计算 RoPE 的逆频率和注意力缩放因子，返回一个包含逆频率张量和缩放因子的元组
        inv_freq, self.attention_scaling = rope_init_fn(self.config, device) 
        
        # 将计算得到的逆频率张量注册为模型的缓冲区，`persistent=False` 表示该缓冲区不会被保存到模型的状态字典中，适用于在推理过程中动态更新 RoPE 参数的情况
        self.register_buffer("inv_freq", inv_freq, persistent=False)
        # 将原始的逆频率张量注册为模型的缓冲区，`persistent=False` 表示该缓冲区不会被保存到模型的状态字典中，适用于在推理过程中动态更新 RoPE 参数的情况
        self.register_buffer("original_inv_freq", inv_freq.clone(), persistent=False)

    @staticmethod
    def compute_default_rope_parameters(
        config: Qwen2Config | None = None,
        device: Optional["torch.device"] = None,
        seq_len: int | None = None,
    ) -> tuple["torch.Tensor", float]:
        """
        依据 RoPE（旋转位置编码）的原始实现逻辑计算逆频率。
        参数：
            config（[`~transformers.PreTrainedConfig`] 类型）：
                模型的配置对象。
            device（`torch.device` 类型）：
                用于初始化逆频率张量的设备（如 CPU/GPU）。
            seq_len（`int` 类型，可选）：
                当前序列长度。该参数对此类 RoPE 实现无实际作用。
        返回值：
            一个包含两个元素的元组 (`torch.Tensor`, `float`)：第一个元素是 RoPE 嵌入所用的逆频率张量，
            第二个元素是应用于计算出的余弦/正弦值的后处理缩放系数（此类 RoPE 实现中未使用该系数）。
        """
        base = config.rope_parameters["rope_theta"]  # RoPE 的基数，决定了位置编码的频率分布，通常是一个大于 1 的实数，较大的基数会导致位置编码的频率更快地增加
        dim = getattr(config, "head_dim", None) or config.hidden_size // config.num_attention_heads

        attention_factor = 1.0  # 注意力缩放因子，默认值为 1.0，表示不进行缩放，在某些 RoPE 实现中可能会使用该因子来调整计算出的余弦/正弦值的范围，以适应不同的模型架构或训练设置

        # 计算 RoPE 的逆频率张量，使用基数和维度来计算每个位置的频率，通常是通过将基数的幂次方与位置索引相结合来计算得到的，结果是一个包含每个位置对应的频率值的张量
        inv_freq = 1.0 / (
            base ** (torch.arange(0, dim, 2, dtype=torch.int64).to(device=device, dtype=torch.float) / dim)
        )
        return inv_freq, attention_factor

    @torch.no_grad()
    @dynamic_rope_update  # 使用 `dynamic_rope_update` 装饰器来实现 RoPE 参数的动态更新功能，当输入序列长度超过当前缓存的最大序列长度时，自动重新计算 RoPE 的逆频率张量以适应新的序列长度
    def forward(self, x, position_ids):
        # 将 RoPE 的逆频率张量扩展到与输入序列长度和批次大小相匹配的形状，以便在计算位置编码时进行广播操作，确保每个位置都能够正确地应用 RoPE 的旋转变换
        inv_freq_expanded = self.inv_freq[None, :, None].float().expand(position_ids.shape[0], -1, 1).to(x.device)
        # 将位置 ID 张量扩展到与 RoPE 的逆频率张量相匹配的形状，以便在计算位置编码时进行广播操作，确保每个位置都能够正确地应用 RoPE 的旋转变换
        position_ids_expanded = position_ids[:, None, :].float() 

        device_type = x.device.type if isinstance(x.device.type, str) and x.device.type != "mps" else "cpu"
        with maybe_autocast(device_type=device_type, enabled=False):  # 强制使用FP32精度
            freqs = (inv_freq_expanded.float() @ position_ids_expanded.float()).transpose(1, 2)  # 计算位置编码的频率值，使用矩阵乘法将扩展后的逆频率张量与扩展后的位置 ID 张量相乘，得到每个位置对应的频率值，然后进行转置以匹配后续计算的形状要求
            emb = torch.cat((freqs, freqs), dim=-1)  # 将频率值张量复制一份并在最后一个维度上进行拼接，得到一个包含正弦和余弦频率值的张量，通常是为了在后续计算中同时使用正弦和余弦函数来生成位置编码
            cos = emb.cos() * self.attention_scaling  # 计算位置编码的余弦值，并应用注意力缩放因子，得到一个包含位置编码余弦值的张量，通常用于在注意力机制中对输入序列进行位置编码的旋转变换
            sin = emb.sin() * self.attention_scaling  # 计算位置编码的正弦值，并应用注意力缩放因子，得到一个包含位置编码正弦值的张量，通常用于在注意力机制中对输入序列进行位置编码的旋转变换

        return cos.to(dtype=x.dtype), sin.to(dtype=x.dtype)

In [ ]:
# 定义一个辅助函数 `rotate_half`，用于对输入的一半隐藏维度进行旋转操作
# 通常在应用 RoPE 的位置编码时使用，通过将输入张量的前一半和后一半进行拼接，并对后一半进行符号反转来实现旋转变换
# 作用是在计算位置编码时能够正确地应用 RoPE 的旋转变换，确保模型能够捕捉到输入序列中不同位置之间的相对关系
def rotate_half(x):
    """对输入的一半隐藏维度进行旋转操作"""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)

In [ ]:
# 使用 `use_kernel_func_from_hub` 装饰器来指定在调用 `apply_rotary_pos_emb` 函数时优先使用从 Hugging Face Hub 加载的优化版本的函数
# 如果该函数不可用，则使用当前模块中定义的版本
@use_kernel_func_from_hub("rotary_pos_emb")  
# 定义 `apply_rotary_pos_emb` 函数，用于将 RoPE 应用到查询和键张量上，计算位置编码的旋转变换
# 并将其应用到输入的查询和键张量中，以生成带有位置编码的查询和键张量
# 通常在注意力机制中使用，以增强模型对输入序列中不同位置之间关系的建模能力
def apply_rotary_pos_emb(q, k, cos, sin, unsqueeze_dim=1):
    """将旋转位置编码（Rotary Position Embedding，RoPE）应用于查询（query）和键（key）张量。

    参数：
        q（`torch.Tensor` 类型）：查询张量。
        k（`torch.Tensor` 类型）：键张量。
        cos（`torch.Tensor` 类型）：旋转位置编码的余弦分量。
        sin（`torch.Tensor` 类型）：旋转位置编码的正弦分量。
        unsqueeze_dim（`int` 类型，可选，默认值为 1）：
            `unsqueeze_dim` 参数指定对 cos[position_ids] 和 sin[position_ids] 进行维度扩展（unsqueeze）的维度方向，
            使其能够正确广播（broadcast）到 q 和 k 的维度。例如：若 cos[position_ids] 和 sin[position_ids] 的形状为
            [batch_size, seq_len, head_dim]，而 q 和 k 的形状为 [batch_size, heads, seq_len, head_dim]，
            则设置 unsqueeze_dim=1 可让 cos/sin 张量广播至与 q、k 相同的形状；同理，若 q 和 k 的形状为
            [batch_size, seq_len, heads, head_dim]，则需将 unsqueeze_dim 设为 2。
    返回值：
        一个包含两个元素的元组（`tuple(torch.Tensor)`），元素分别为经过旋转位置编码旋转后的查询张量和键张量。
    """
    cos = cos.unsqueeze(unsqueeze_dim)
    sin = sin.unsqueeze(unsqueeze_dim)
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed

---

#### 一、多头注意力（MHA：Multi-Head Attention）
##### 核心定义
Transformer原生注意力机制（《Attention Is All You Need》提出），Q、K、V各有相同的H个独立头，每个头独立计算注意力，最后拼接结果，是**表达能力最强**的版本，也是MQA/GQA的优化基础。
##### 执行步骤
假设输入为$X \in \mathbb{R}^{N \times 512}$（$N$为序列长度，如句子的token数）
1. **独立投影**：用3个**独立的权重矩阵**$W_Q \in \mathbb{R}^{512 \times 512}$、$W_K \in \mathbb{R}^{512 \times 512}$、$W_V \in \mathbb{R}^{512 \times 512}$，对X做线性投影，得到**同头数**的Q、K、V：
   $Q = XW_Q \in \mathbb{R}^{N \times 512}$，$K = XW_K \in \mathbb{R}^{N \times 512}$，$V = XW_V \in \mathbb{R}^{N \times 512}$；
2. **分拆成头**：将Q、K、V各拆为$H=8$个独立的头，每个头维度为$d_k=64$，得到：
   $Q_h \in \mathbb{R}^{8 \times N \times 64}$，$K_h \in \mathbb{R}^{8 \times N \times 64}$，$V_h \in \mathbb{R}^{8 \times N \times 64}$（维度：头数×序列长度×头维度）；
3. **独立计算注意力**：8个头**互不干扰**，每个头单独执行注意力公式，得到8个注意力输出：
   $head_i = Attention(Q_i, K_i, V_i) \in \mathbb{R}^{N \times 64}$（$i=1,2,...,8$）；
4. **拼接头输出**：将8个独立的头结果按最后一维拼接，恢复为$d_{model}$维度：
   $Concat = [head_1, head_2, ..., head_8] \in \mathbb{R}^{N \times 512}$；
5. **最终投影**：用输出权重矩阵$W_O \in \mathbb{R}^{512 \times 512}$做线性变换，得到MHA最终结果：
   $Output = Concat \cdot W_O \in \mathbb{R}^{N \times 512}$。

##### 关键开销（示例参数）
- 投影参数：$W_Q+W_K+W_V = 3×512×512 = 786432$ 个参数；
- KV Cache内存：每层需缓存$8$个K和$8$个V，单层缓存规模为$2×8×N×64 = 1024N$（序列长度N越大，开销越高）；
- 注意力计算量：$8×(2×N×64×N + N×64×N) = 8×3N^2×64 = 1536N^2$（与头数成正比）。

##### 核心特点
✅ **表达能力最强**：8个头独立捕捉不同的语义/位置特征（如句法关系、语义关联），适配复杂任务；  
❌ **推理效率最低**：KV Cache内存占用高，长序列（如N=4096/8192）下内存压力大，硬件带宽消耗高；  
❌ **参数冗余**：部分任务中，多个头的特征存在重叠，完全独立的头设计造成计算/内存浪费。 

#### 二、多查询注意力（MQA：Multi-Query Attention）
##### 核心定义
为极致优化推理效率提出的变体（最早见于《Fast Transformer Decoding: One Write-Head is All You Need》），保留H个独立的Q头，但仅用1个共享的K头和1个共享的V头，所有Q头共享同一组K、V计算注意力，是**效率最高、表达能力最弱**的版本。
##### 执行步骤（基于MHA同参数：$d_{model}=512$、$H=8$、$d_k=64$）
输入仍为$X \in \mathbb{R}^{N \times 512}$，核心变化是**K/V的投影矩阵和头数**：
1. **非对称投影**：Q用**多头投影矩阵**，K/V用**单头投影矩阵**，维度匹配是关键：
   - Q投影：$W_Q \in \mathbb{R}^{512 \times 512}$（与MHA一致），$Q = XW_Q \in \mathbb{R}^{N \times 512}$；
   - K投影：$W_K \in \mathbb{R}^{512 \times 64}$（仅单头维度$d_k$），$K = XW_K \in \mathbb{R}^{N \times 64}$；
   - V投影：$W_V \in \mathbb{R}^{512 \times 64}$（仅单头维度$d_k$），$V = XW_V \in \mathbb{R}^{N \times 64}$；
2. **分拆Q头，K/V保持单头**：
   - Q拆为8个独立头：$Q_h \in \mathbb{R}^{8 \times N \times 64}$（与MHA一致）；
   - K/V不拆分：$K \in \mathbb{R}^{N \times 64}$，$V \in \mathbb{R}^{N \times 64}$（无头部维度）；
3. **共享KV计算注意力**：8个Q头**全部使用同一组K、V**，各自独立计算注意力，得到8个输出：
   $head_i = Attention(Q_i, K, V) \in \mathbb{R}^{N \times 64}$（$i=1,2,...,8$）；
4. **拼接+最终投影**：与MHA完全一致，拼接8个头→$N×512$，再通过$W_O$得到最终输出。

##### 关键开销（示例参数）
- 投影参数：$W_Q + W_K + W_V = 512×512 + 2×512×64 = 512×(512+128) = 327680$ 个参数（仅为MHA的~41.6%）；
- KV Cache内存：每层仅缓存**1个K和1个V**，单层缓存规模为$2×1×N×64 = 128N$（仅为MHA的1/8，长序列下优势极致）；
- 注意力计算量：$8×(2×N×64×N + N×64×N) = 1536N^2$（与MHA一致，计算量未减少，优化的是**内存/带宽**）。

##### 核心特点
✅ **推理效率极致**：KV Cache内存占用降至MHA的$1/H$，大幅降低硬件带宽消耗，长序列推理速度提升显著；  
✅ **参数更少**：K/V投影矩阵维度大幅减小，模型总参数量降低；  
❌ **表达能力损失大**：所有Q头共享同一组K/V，无法捕捉多样化的特征关联，复杂任务（如大模型生成、精细语义理解）中**性能明显下降**；  
❌ **泛化性弱**：单组KV难以适配不同Q头的特征需求，易出现特征混叠。

#### 三、分组查询注意力（GQA：Grouped-Query Attention）
##### 核心定义
2023年《GQA: Training Generalized Multi-Query Transformer Models from Multi-Head Checkpoints》提出的中间折中方案，融合MHA的表达能力和MQA的推理效率，将H个Q头划分为G个组，**每组内的Q头共享1组独立的K/V头**（G为组数，可灵活调整）。
##### 核心参数关系（统一约束）
- 总Q头数：$H$（固定，由$d_{model}/d_k$决定）；
- 组数：$G$（超参数，人工设定，如2/4/8）；
- 每组Q头数：$h = H/G$（必须为整数，如$H=8$、$G=4$时，$h=2$）；
- K/V头数：**等于组数G**（每组1组独立KV，无共享）。

**示例参数**（延续前文，且选工程中常用的$G=4$）：$d_{model}=512$、$H=8$、$G=4$、$h=2$、$d_k=64$。

##### 执行步骤（G=4为示例）
输入仍为$X \in \mathbb{R}^{N \times 512}$，核心是**分组划分+组内KV共享**：
1. **分组投影**：Q用多头投影矩阵，K/V用**组数G对应的投影矩阵**（头数=G）：
   - Q投影：$W_Q \in \mathbb{R}^{512 \times 512}$（与MHA一致），$Q = XW_Q \in \mathbb{R}^{N \times 512}$；
   - K投影：$W_K \in \mathbb{R}^{512 \times (G×d_k)} = \mathbb{R}^{512 \times 256}$，$K = XW_K \in \mathbb{R}^{N \times 256}$；
   - V投影：$W_V \in \mathbb{R}^{512 \times (G×d_k)} = \mathbb{R}^{512 \times 256}$，$V = XW_V \in \mathbb{R}^{N \times 256}$；
2. **分拆Q头，分拆KV为组**：
   - Q拆为8个独立头：$Q_h \in \mathbb{R}^{8 \times N \times 64}$，并按$G=4$分组：$[Q_1Q_2, Q_3Q_4, Q_5Q_6, Q_7Q_8]$（每组2个Q头）；
   - K拆为G=4个组头：$K_g \in \mathbb{R}^{4 \times N \times 64}$（每组1个K头，对应1组Q头）；
   - V拆为G=4个组头：$V_g \in \mathbb{R}^{4 \times N \times 64}$（与K头一一对应）；
3. **组内共享KV，组间独立计算**：**不同组之间KV完全独立，同组内所有Q头共享本组KV**，4个组并行计算注意力：
   - 组1：$head_1=Attention(Q_1,K_1,V_1)$、$head_2=Attention(Q_2,K_1,V_1)$；
   - 组2：$head_3=Attention(Q_3,K_2,V_2)$、$head_4=Attention(Q_4,K_2,V_2)$；
   - 组3/4：按上述规则执行，最终得到8个注意力头输出；
4. **拼接+最终投影**：与MHA、MQA完全一致，拼接8个头→$N×512$，通过$W_O$得到最终输出。

##### 关键开销（示例参数：G=4）
- 投影参数：$W_Q + W_K + W_V = 512×512 + 2×512×256 = 512×(512+512) = 524288$ 个参数（介于MHA和MQA之间）；
- KV Cache内存：每层缓存**G=4个K和4个V**，单层缓存规模为$2×4×N×64 = 512N$（为MHA的1/2、MQA的4倍，可通过调整G灵活控制）；
- 注意力计算量：$8×3N^2×64 = 1536N^2$（与MHA、MQA一致，计算量无损失，仅优化内存/带宽）。

##### 灵活调优性（G的核心作用）
GQA的精髓在于**组数G是可调节的超参数**，可实现「效率-性能」的连续权衡：
- 当$G=H$（如G=8）：每组1个Q头，K/V头数=Q头数，**GQA等价于MHA**（表达能力拉满，效率最低）；
- 当$G=1$：所有Q头归为1组，共享1组KV，**GQA等价于MQA**（效率拉满，表达能力最弱）；
- 当$1<G<H$（如G=2/4/6）：介于两者之间，是工程中最常用的选择（如GPT-4采用GQA，G取8/16）。

##### 核心特点
✅ **效率与性能平衡**：在损失少量表达能力的前提下，将KV Cache开销降至MHA的$G/H$，长序列推理效率大幅提升，且性能远优于MQA；  
✅ **灵活可控**：通过调整G适配不同硬件（低内存硬件选小G，高内存硬件选大G）和任务（简单任务选小G，复杂任务选大G）；  
✅ **无缝迁移**：可直接从MHA预训练模型的权重微调得到GQA模型，无需从零训练，工程成本低；  
✅ **计算量无损失**：仅优化KV的存储和带宽，注意力计算量与MHA一致，不影响训练/推理的计算速度；  
❌ **存在轻微表达损失**：相比MHA，组内Q头共享KV会损失少量特征多样性，但工程中可通过合理选G将损失降至可接受范围（如G=4时，性能与MHA几乎持平）。

#### 四、三者核心维度对比表（统一参数：H=8、d_k=64、G=4）
为了更直观的对比，将三者的核心差异汇总为表格，**红色为核心差异点**：

|对比维度|多头注意力（MHA）|多查询注意力（MQA）|分组查询注意力（GQA）|
| ---- | ---- | ---- | ---- |
|Q头数|8（独立）|8（独立）|8（独立，分4组）|
|**K/V头数**|**8（与Q一一对应）**|**1（所有Q共享）**|**4（与组数G一致，组内共享）**|
|投影参数规模|786432（最大）|327680（最小）|524288（中间）|
|**KV Cache内存（单层）**|**1024N（最大）**|**128N（最小）**|**512N（中间）**|
|注意力计算量|$1536N^2$|$1536N^2$|$1536N^2$（三者一致）|
|表达能力|⭐⭐⭐⭐⭐（最强）|⭐⭐（最弱）|⭐⭐⭐⭐（接近MHA）|
|推理效率（内存/带宽）|⭐（最低）|⭐⭐⭐⭐⭐（最高）|⭐⭐⭐⭐（接近MQA）|
|参数冗余度|高（头间特征重叠）|无（单KV）|中（组内无重叠，组间少量重叠）|
|工程适配性|适合训练/小序列推理|适合极致效率的简单任务|**大模型长序列推理（最优选择）**|
|典型应用|Transformer基础版、小模型|轻量级端侧模型、快速推理场景|GPT-4、PaLM、LLaMA2（大模型主流）|

#### 五、核心总结
1. **设计初衷**：MHA是基础，追求表达能力；MQA为极致效率牺牲表达；GQA是工程化的最优折中，解决大模型长序列推理的「内存瓶颈」；
2. **核心优化点**：三者的**注意力计算量完全一致**，MQA/GQA的优化核心是**减少KV Cache的内存占用和硬件带宽消耗**（推理阶段的核心开销），而非计算速度；
3. **GQA的核心价值**：成为大模型的主流选择，本质是解决了「大模型长序列推理时，MHA内存不够用，MQA性能损失太大」的工程痛点；
4. **选型原则**：
   - 训练阶段/小序列推理/复杂任务：选MHA，保证表达能力；
   - 端侧模型/简单任务/极致效率需求：选MQA；
   - 大模型/长序列推理（如N=4096/8192）/兼顾性能与效率：选GQA（重点调优组数G）。


---

In [ ]:
# 定义 `repeat_kv` 函数，用于重复键值张量以匹配注意力头的数量，通常在使用 RoPE 时需要将键值张量重复以适应不同的注意力头
# 常见于GQA（Gated Query Attention）等注意力机制中，其中查询、键和值的头数可能不同，因此需要对键值张量进行重复以匹配查询的头数
def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    此操作等效于执行 `torch.repeat_interleave(x, dim=1, repeats=n_rep)`。
    隐藏状态张量的形状会从 `(batch, num_key_value_heads, seqlen, head_dim)`变为 `(batch, num_attention_heads, seqlen, head_dim)`
    """
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape
    if n_rep == 1:
        return hidden_states
    hidden_states = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    return hidden_states.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)

In [ ]:
# 定义 `eager_attention_forward` 函数，用于实现 Qwen2 模型中的注意力机制的前向传播逻辑，包括计算注意力权重和生成注意力输出等功能
def eager_attention_forward(
    module: nn.Module,
    query: torch.Tensor,
    key: torch.Tensor,
    value: torch.Tensor,
    attention_mask: torch.Tensor | None,
    scaling: float,  # 注意力缩放因子，通常是一个小于 1 的实数，用于在计算注意力权重时对点积结果进行缩放，以防止数值过大导致梯度消失或爆炸
    dropout: float = 0.0,  # 注意力机制中的 dropout 概率，用于在训练过程中随机丢弃一些注意力权重，以防止过拟合
    **kwargs: Unpack[TransformersKwargs],
):
    key_states = repeat_kv(key, module.num_key_value_groups)
    value_states = repeat_kv(value, module.num_key_value_groups)

    # 计算注意力权重，使用查询张量与键张量的转置进行矩阵乘法，并应用注意力缩放因子进行缩放，以得到每个查询与所有键之间的相似度分数
    attn_weights = torch.matmul(query, key_states.transpose(2, 3)) * scaling
    # 如果提供了注意力掩码，则将其添加到注意力权重中，以屏蔽掉不需要关注的位置
    # 通常是通过将掩码中的无效位置设置为一个很大的负数来实现的，这样在后续的 softmax 操作中这些位置的权重将接近于零
    if attention_mask is not None:
        attn_weights = attn_weights + attention_mask

    attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(query.dtype)
    # 在训练过程中应用 dropout，以随机丢弃一些注意力权重，防止过拟合，同时保持数值稳定性
    attn_weights = nn.functional.dropout(attn_weights, p=dropout, training=module.training)
    attn_output = torch.matmul(attn_weights, value_states)
    # 将注意力输出的维度进行转置和连续化处理，以匹配后续层的输入要求
    # 形状转换为 (batch_size, seq_len, num_attention_heads, head_dim)
    attn_output = attn_output.transpose(1, 2).contiguous() 

    return attn_output, attn_weights

In [ ]:
# 使用 `use_kernelized_func` 装饰器来指定在调用 `Qwen2Attention` 类的前向传播方法时优先使用从 Hugging Face Hub 加载的优化版本的 `apply_rotary_pos_emb` 函数
# 如果该函数不可用，则使用当前模块中定义的版本
@use_kernelized_func(apply_rotary_pos_emb)  
# 定义 Qwen2Attention 类，继承自 nn.Module，负责实现 Qwen2 模型中的多头注意力机制，包括查询、键、值的线性变换和位置编码的应用等功能
class Qwen2Attention(nn.Module):
    """来自论文 Attention Is All You Need 的多头注意力机制"""

    def __init__(self, config: Qwen2Config, layer_idx: int):
        super().__init__()
        self.layer_type = config.layer_types[layer_idx] if hasattr(config, "layer_types") else None
        self.config = config
        self.layer_idx = layer_idx
        # 每个注意力头的维度，通常是隐藏层维度除以注意力头的数量，如果配置中直接提供了 `head_dim` 参数，则使用该参数的值
        self.head_dim = getattr(config, "head_dim", config.hidden_size // config.num_attention_heads) 
        # 注意力头分组的数量，通常是注意力头的数量除以键值头的数量，如果配置中直接提供了 `num_key_value_groups` 参数，则使用该参数的值
        self.num_key_value_groups = config.num_attention_heads // config.num_key_value_heads
        # 注意力缩放因子，通常是每个注意力头的维度的平方根的倒数，用于在计算注意力权重时对点积结果进行缩放，以防止数值过大导致梯度消失或爆炸
        self.scaling = self.head_dim**-0.5
        self.attention_dropout = config.attention_dropout
        self.is_causal = True  # 是否使用因果注意力机制，通常在语言模型中使用，以确保模型只能关注当前时间步之前的输入，防止信息泄露
        self.q_proj = nn.Linear(config.hidden_size, config.num_attention_heads * self.head_dim, bias=True)
        self.k_proj = nn.Linear(config.hidden_size, config.num_key_value_heads * self.head_dim, bias=True)
        self.v_proj = nn.Linear(config.hidden_size, config.num_key_value_heads * self.head_dim, bias=True)
        self.o_proj = nn.Linear(config.num_attention_heads * self.head_dim, config.hidden_size, bias=False)
        # 滑动窗口注意力：仅当层类型为 sliding_attention 时生效（限制注意力范围，提升效率）
        self.sliding_window = config.sliding_window if self.layer_type == "sliding_attention" else None

    def forward(
        self,
        hidden_states: torch.Tensor,
        position_embeddings: tuple[torch.Tensor, torch.Tensor],
        attention_mask: torch.Tensor | None,
        past_key_values: Cache | None = None,
        cache_position: torch.LongTensor | None = None,
        **kwargs: Unpack[FlashAttentionKwargs],
    ) -> tuple[torch.Tensor, torch.Tensor | None]:
        input_shape = hidden_states.shape[:-1]  # 获取输入隐藏状态的形状，通常是 (batch_size, seq_len)
        hidden_shape = (*input_shape, -1, self.head_dim)  # 计算查询、键、值张量的形状，通常是 (batch_size, seq_len, num_attention_heads, head_dim)

        # 形状变换为 (batch_size, num_attention_heads, seq_len, head_dim)
        query_states = self.q_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        # 形状变换为 (batch_size, num_key_value_heads, seq_len, head_dim)
        key_states = self.k_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        # 形状变换为 (batch_size, num_key_value_heads, seq_len, head_dim)
        value_states = self.v_proj(hidden_states).view(hidden_shape).transpose(1, 2)

        cos, sin = position_embeddings
        # 得到应用了 RoPE 位置编码旋转变换后的查询和键张量，增强模型对输入序列中不同位置之间关系的建模能力
        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin)

        if past_key_values is not None:
            # 正弦值（sin）和余弦值（cos）是 RoPE 类模型特有的参数；静态缓存（static cache）的使用则需要依赖缓存位置（cache_position）这一参数
            cache_kwargs = {"sin": sin, "cos": cos, "cache_position": cache_position}
            # 更新键值张量以包含当前时间步的键值对，同时根据缓存位置（cache_position）来确定如何更新缓存中的键值对，以支持在生成过程中逐步构建缓存
            key_states, value_states = past_key_values.update(key_states, value_states, self.layer_idx, cache_kwargs)

        # 根据配置中的注意力实现方式选择相应的注意力接口函数，默认使用 `eager_attention_forward` 函数来计算注意力权重和生成注意力输出
        # 如果配置中指定了其他注意力实现方式，则从 `ALL_ATTENTION_FUNCTIONS` 字典中获取对应的函数
        attention_interface: Callable = ALL_ATTENTION_FUNCTIONS.get_interface(
            self.config._attn_implementation, eager_attention_forward
        )

        attn_output, attn_weights = attention_interface(
            self,
            query_states,
            key_states,
            value_states,
            attention_mask,
            dropout=0.0 if not self.training else self.attention_dropout,
            scaling=self.scaling,
            sliding_window=self.sliding_window,  # main diff with Llama
            **kwargs,
        )

        # 将注意力输出的形状转换回 (batch_size, seq_len, hidden_size)，以匹配后续层的输入要求
        attn_output = attn_output.reshape(*input_shape, -1).contiguous()
        attn_output = self.o_proj(attn_output)  # 通过输出投影层将注意力输出映射回隐藏层的维度，得到最终的注意力输出张量
        return attn_output, attn_weights

### 均方根层归一化 (Root Mean Square Layer Normalization, RMSNorm)

RMSNorm 是对 LayerNorm 的一个改进,  没有做 re-center 操作（移除了均值项）, 可以看作 LayerNorm 在均值为零时的特例, 使用平方根均值归一化降低噪声影响。

- **Layer Norm**

$$y = \frac{x-E(x)}{\sqrt{Var(x) + \epsilon}} * \gamma + \beta$$

假设输入张量形状为 (batch_size,  sequence_length,  embedding_dim), 层归一化对 embedding_dim 维度进行归一化操作, 其中,  $\epsilon$ 是一个超参数, 用于防止分母为零导致结果上溢,  $\gamma$,  $\beta$ 均为可学习参数。

- **RMS Norm**

$$y=\frac{x}{\sqrt{\frac{1}{n}\sum^n_{i=1}x^2_i + \epsilon}} * \gamma$$

假设输入张量形状为 (batch_size,  sequence_length,  embedding_dim), RMS Norm 对 embedding_dim 维度进行归一化,其中,  其中,  $\epsilon$ 是一个超参数, 用于防止分母为零导致结果上溢, $\gamma$ 为可学习参数.

不难发现, 当均值为零时, Layer Norm 退化为 RMS Norm. 这是因为 RMS Norm 在 Layer Norm 的基础上舍弃了中心化操作, 仅用缩放进行归一化, 其不改变数据原本的分布, 有利于激活函数输出的稳定.

---

#### RMSNorm 相比 LayerNorm 的核心优势
##### 1. 计算更高效，推理/训练速度更快
- **减少计算量**：去掉了「均值计算」和「均值中心化」步骤，仅需计算均方根（RMS），浮点运算（FLOPs）减少约 15%-25%。
- **内存访问更友好**：无需存储和计算均值，减少了中间变量的读写，尤其在大模型（如 7B/13B 参数）中，推理延迟和训练耗时的优化更明显。
- 实际效果：在同等硬件下，RMSNorm 能让 Transformer 训练/推理速度提升 5%-15%，且不损失模型性能。

##### 2. 简化参数，降低过拟合风险
- LayerNorm 有两组可学习参数（$ \gamma, \beta $），RMSNorm 仅保留 $ \gamma $，参数总量减少约一半（归一化层维度通常等于模型隐层维度，如 768/4096，累计参数节省可观）。
- 更少的参数意味着模型更难「过拟合」训练数据，尤其在小样本训练或模型轻量化场景中，泛化能力更稳定。

##### 3. 缓解均值中心化的副作用，训练更稳定
LayerNorm 的「均值中心化」会强制让输入特征的均值为 0，但在 Transformer 的残差连接（Residual Connection）中，残差流的均值会被反复扰动，可能导致：
- 梯度传播不稳定（尤其是深层模型）；
- 激活值分布被过度约束，影响模型表达能力。

RMSNorm 去掉均值中心化，仅通过 RMS 缩放控制激活值的**尺度**，保留了特征的原始分布信息，让残差流更平滑，深层模型的训练稳定性显著提升（这也是 GPT-3、LLaMA 等大模型选择 RMSNorm 的关键原因）。

##### 4. 对异常值更鲁棒
RMS 基于「平方和的平均」计算，相比 LayerNorm 的「方差（基于均值偏差）」，对极端异常值的敏感度更低。在自然语言处理中，文本序列的 token 分布往往存在长尾特征，RMSNorm 能更稳定地约束激活值范围，避免异常值导致的训练震荡。

##### 5. 与 Transformer 架构的兼容性更好
Transformer 的核心是「自注意力+前馈网络+残差连接」，RMSNorm 的设计更贴合残差流的特性：
- 不破坏残差连接的恒等映射特性（均值中心化会轻微改变残差流的恒等性）；
- 与 SwiGLU、GELU 等现代激活函数配合更默契，能进一步提升模型收敛速度和最终效果。

---

In [ ]:
# 使用 `use_kernel_forward_from_hub` 装饰器优先使用从 Hugging Face Hub 加载的优化版本的 RMSNorm 函数
@use_kernel_forward_from_hub("RMSNorm")  
# 定义 Qwen2RMSNorm 类，继承自 nn.Module，负责实现 Qwen2 模型中的 RMSNorm（Root Mean Square Normalization）功能
# 包括计算输入张量的均方根并进行归一化处理等功能
class Qwen2RMSNorm(nn.Module):
    def __init__(self, hidden_size, eps: float = 1e-6) -> None:
        """
        Qwen2RMSNorm 归一化层等效于 T5LayerNorm 归一化层
        """
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        input_dtype = hidden_states.dtype
        hidden_states = hidden_states.to(torch.float32)
        variance = hidden_states.pow(2).mean(-1, keepdim=True)
        hidden_states = hidden_states * torch.rsqrt(variance + self.variance_epsilon)
        return self.weight * hidden_states.to(input_dtype)

    def extra_repr(self):
        return f"{tuple(self.weight.shape)}, eps={self.variance_epsilon}"

In [ ]:
# 定义 Qwen2DecoderLayer 类，继承自 GradientCheckpointingLayer，负责实现 Qwen2 模型中的解码器层功能
# 包括多头注意力机制、前馈网络和层归一化等功能
class Qwen2DecoderLayer(GradientCheckpointingLayer):
    def __init__(self, config: Qwen2Config, layer_idx: int):
        super().__init__()
        self.hidden_size = config.hidden_size
        # 根据配置中的层类型列表和当前层索引来确定该层使用的注意力机制类型，通常是 "full_attention"（全局注意力）或 "sliding_attention"（滑动窗口注意力）
        self.self_attn = Qwen2Attention(config=config, layer_idx=layer_idx)

        self.mlp = Qwen2MLP(config)
        # 输入层归一化，通常在注意力机制之前应用，以确保输入的隐藏状态具有适当的分布，增强模型的训练稳定性和性能
        self.input_layernorm = Qwen2RMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        # 注意力输出层归一化，通常在前馈网络之前应用，以确保注意力输出的隐藏状态具有适当的分布，增强模型的训练稳定性和性能
        self.post_attention_layernorm = Qwen2RMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.attention_type = config.layer_types[layer_idx]  # 记录该层的注意力类型

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: torch.Tensor | None = None,
        position_ids: torch.LongTensor | None = None,
        past_key_values: Cache | None = None,
        use_cache: bool | None = False,
        cache_position: torch.LongTensor | None = None,
        position_embeddings: tuple[torch.Tensor, torch.Tensor] | None = None,
        **kwargs: Unpack[TransformersKwargs],
    ) -> torch.Tensor:
        residual = hidden_states  # 保存输入的隐藏状态作为残差连接的一部分，以便在注意力机制和前馈网络之后将其与输出进行相加，增强模型的训练稳定性和性能
        hidden_states = self.input_layernorm(hidden_states)  # 对输入的隐藏状态进行层归一化处理，确保其具有适当的分布，增强模型的训练稳定性和性能
        # 自注意力层
        hidden_states, _ = self.self_attn(
            hidden_states=hidden_states,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            use_cache=use_cache,
            cache_position=cache_position,
            position_embeddings=position_embeddings,
            **kwargs,
        )
        hidden_states = residual + hidden_states  # 将注意力机制的输出与输入的隐藏状态进行残差连接

        # 全连接层
        residual = hidden_states  
        hidden_states = self.post_attention_layernorm(hidden_states) 
        hidden_states = self.mlp(hidden_states)
        hidden_states = residual + hidden_states
        return hidden_states

In [ ]:
# 自动生成文档字符串的装饰器，用于在类或函数定义时自动生成相应的文档字符串，增强代码的可读性和可维护性
@auto_docstring 
# 定义 Qwen2PreTrainedModel 类，继承自 PreTrainedModel，负责实现 Qwen2 模型的预训练模型功能，包括模型配置、前向传播和其他相关功能
class Qwen2PreTrainedModel(PreTrainedModel):
    config: Qwen2Config
    base_model_prefix = "model"  # 模型的前缀名称，通常用于在保存和加载模型时指定模型权重的命名空间
    supports_gradient_checkpointing = True  # 是否支持梯度检查点功能，允许在训练过程中节省内存，但可能会增加计算时间
    _no_split_modules = ["Qwen2DecoderLayer"]  # 在模型并行训练中不进行切分的模块列表，通常是一些包含多个子模块的层，如解码器层，以确保它们在训练过程中能够保持完整性
    _skip_keys_device_placement = ["past_key_values"]  # 在模型并行训练中跳过设备放置的键列表，通常是一些包含缓存键值对的参数，如 `past_key_values`，以确保它们在训练过程中能够正确地放置在设备上
    _supports_flash_attn = True  # 是否支持 Flash Attention，一种优化的注意力机制实现，可以显著提高训练和推理的效率，特别是在处理长序列时
    _supports_sdpa = True  # 是否支持 SDPA（Scaled Dot-Product Attention），一种常见的注意力机制实现，适用于各种模型架构和训练设置
    _supports_flex_attn = True  # 是否支持 Flex Attention，一种灵活的注意力机制实现，可以根据输入序列的长度和模型的配置动态调整注意力计算的方式，以提高效率和性能

    _can_compile_fullgraph = True  # 是否支持完整图编译，允许在训练和推理过程中将整个模型编译成一个优化的计算图，以提高效率和性能
    _supports_attention_backend = True  # 是否支持指定注意力后端，允许用户在训练和推理过程中选择不同的注意力实现方式，以适应不同的模型架构和训练设置
    _can_record_outputs = {
        "hidden_states": Qwen2DecoderLayer,  # 指定在记录模型输出时，指定 `Qwen2DecoderLayer` 模块的输出将被记录为 `hidden_states`
        "attentions": Qwen2Attention,  # 指定在记录模型输出时，指定 `Qwen2Attention` 模块的输出将被记录为 `attentions`
    }


In [ ]:
@auto_docstring 
# 定义 Qwen2Model 类，继承自 Qwen2PreTrainedModel，负责实现 Qwen2 模型的具体架构和前向传播逻辑，包括嵌入层、解码器层和输出层等功能
class Qwen2Model(Qwen2PreTrainedModel):
    def __init__(self, config: Qwen2Config):
        super().__init__(config)
        self.padding_idx = config.pad_token_id
        self.vocab_size = config.vocab_size
        # 嵌入层，将输入的 token ID 映射到隐藏层的维度，通常是一个词嵌入矩阵，用于将离散的 token ID 转换为连续的向量表示
        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size, self.padding_idx)  
        # 解码器层列表，根据配置中的层数和层类型来创建相应数量的解码器层，每个解码器层都包含多头注意力机制、前馈网络和层归一化等功能
        self.layers = nn.ModuleList(
            [Qwen2DecoderLayer(config, layer_idx) for layer_idx in range(config.num_hidden_layers)]
        )
        # 输出层归一化，通常在解码器层的输出之后应用，以确保输出的隐藏状态具有适当的分布，增强模型的训练稳定性和性能
        self.norm = Qwen2RMSNorm(config.hidden_size, eps=config.rms_norm_eps) 
        # 旋转位置编码模块，负责计算 RoPE 的逆频率和应用 RoPE 的前向传播逻辑，以增强模型对输入序列中不同位置之间关系的建模能力
        self.rotary_emb = Qwen2RotaryEmbedding(config=config)
        self.gradient_checkpointing = False  # 是否启用梯度检查点功能，允许在训练过程中节省内存，但可能会增加计算时间
        # 滑动窗口注意力层的标志，根据配置中的层类型列表来确定模型是否包含滑动窗口注意力层
        self.has_sliding_layers = "sliding_attention" in self.config.layer_types

        # 初始化模型权重，通常是通过调用 `init_weights` 方法来对模型的参数进行初始化，以确保模型在训练开始时具有适当的权重分布
        self.post_init()

    @merge_with_config_defaults  # 使用 `merge_with_config_defaults` 装饰器来将前向传播方法的输入参数与模型配置中的默认值进行合并
    @capture_outputs  # 使用 `capture_outputs` 装饰器来指定在前向传播过程中捕获模型的输出，以便在需要时进行记录或分析
    @auto_docstring  # 使用 `auto_docstring` 装饰器来自动生成前向传播方法的文档字符串，增强代码的可读性和可维护性
    def forward(
        self,
        input_ids: torch.LongTensor | None = None,
        attention_mask: torch.Tensor | None = None,
        position_ids: torch.LongTensor | None = None,
        past_key_values: Cache | None = None,
        inputs_embeds: torch.FloatTensor | None = None,
        use_cache: bool | None = None,
        cache_position: torch.LongTensor | None = None,
        **kwargs: Unpack[TransformersKwargs],
    ) -> BaseModelOutputWithPast:
        # 确保只能传入 input_ids（token ID 序列）或 inputs_embeds（预计算的嵌入向量）中的一个
        if (input_ids is None) ^ (inputs_embeds is not None):
            raise ValueError("You must specify exactly one of input_ids or inputs_embeds")

        # 如果没有提供输入嵌入张量，则使用嵌入层将输入的 token ID 映射到隐藏层的维度，得到输入嵌入张量
        if inputs_embeds is None:
            inputs_embeds = self.embed_tokens(input_ids)  

        # 如果启用了缓存功能但没有提供 `past_key_values`，则创建一个新的动态缓存实例，用于存储在生成过程中逐步构建的键值对
        if use_cache and past_key_values is None:
            past_key_values = DynamicCache(config=self.config)

        # 如果启用了缓存功能但没有提供 `cache_position`，则根据 `past_key_values` 中已经看到的 token 数量来计算当前的缓存位置
        if cache_position is None:
            past_seen_tokens = past_key_values.get_seq_length() if past_key_values is not None else 0
            cache_position = torch.arange(
                past_seen_tokens, past_seen_tokens + inputs_embeds.shape[1], device=inputs_embeds.device
            )

        # 如果没有提供位置 ID，则使用缓存位置作为位置 ID，以便在计算 RoPE 位置编码时能够正确地应用旋转变换
        if position_ids is None:
            position_ids = cache_position.unsqueeze(0)

        # 该参数可能已由`generate`函数等模块提前生成
        if not isinstance(causal_mask_mapping := attention_mask, dict):
            # 准备生成因果掩码和滑动窗口掩码所需的参数
            mask_kwargs = {
                "config": self.config,
                "inputs_embeds": inputs_embeds,
                "attention_mask": attention_mask,
                "cache_position": cache_position,
                "past_key_values": past_key_values,
                "position_ids": position_ids,
            }
            # 生成掩码
            causal_mask_mapping = {
                "full_attention": create_causal_mask(**mask_kwargs),  # 全注意力因果掩码（防止前瞻）
            }
            # 滑动窗口掩码（仅关注最近 N 个 token，优化长文本）
            if self.has_sliding_layers:
                causal_mask_mapping["sliding_attention"] = create_sliding_window_causal_mask(**mask_kwargs)

        hidden_states = inputs_embeds
        position_embeddings = self.rotary_emb(hidden_states, position_ids)  # 计算 RoPE 位置编码

        # 逐层执行解码器计算
        for decoder_layer in self.layers[: self.config.num_hidden_layers]:
            hidden_states = decoder_layer(
                hidden_states,
                attention_mask=causal_mask_mapping[decoder_layer.attention_type],
                position_embeddings=position_embeddings,
                position_ids=position_ids,
                past_key_values=past_key_values,
                use_cache=use_cache,
                cache_position=cache_position,
                **kwargs,
            )

        hidden_states = self.norm(hidden_states)  # 输出层归一化
        return BaseModelOutputWithPast(
            last_hidden_state=hidden_states,  # 最终的隐藏状态（核心输出）
            past_key_values=past_key_values if use_cache else None,  # 返回缓存（供后续生成使用）
        )

In [ ]:
@auto_docstring
# 定义 Qwen2ForCausalLM 类，继承自 Qwen2PreTrainedModel 和 GenerationMixin，负责实现 Qwen2 模型的语言建模功能
# 包括前向传播和文本生成等功能
class Qwen2ForCausalLM(Qwen2PreTrainedModel, GenerationMixin):
    # 指定语言建模头（lm_head）的权重与嵌入层（embed_tokens）的权重共享，以减少模型参数的数量并增强模型的泛化能力
    _tied_weights_keys = {"lm_head.weight": "model.embed_tokens.weight"}  
    # 定义模型并行训练的切分计划，指定在模型并行训练中如何切分语言建模头（lm_head）模块，以确保其在训练过程中能够正确地分布在不同的设备上
    _tp_plan = {"lm_head": "colwise_gather_output"}
    # 定义完整图编译的输入输出映射，指定在完整图编译过程中语言建模头（lm_head）的输入和输出，以确保在编译过程中能够正确地处理该模块的计算逻辑
    _pp_plan = {"lm_head": (["hidden_states"], ["logits"])}

    def __init__(self, config):
        super().__init__(config)
        self.model = Qwen2Model(config)
        self.vocab_size = config.vocab_size
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

        # 初始化权重并执行最终的处理操作
        self.post_init()

    @can_return_tuple
    @auto_docstring
    def forward(
        self,
        input_ids: torch.LongTensor | None = None,
        attention_mask: torch.Tensor | None = None,
        position_ids: torch.LongTensor | None = None,
        past_key_values: Cache | None = None,
        inputs_embeds: torch.FloatTensor | None = None,
        labels: torch.LongTensor | None = None,
        use_cache: bool | None = None,
        cache_position: torch.LongTensor | None = None,
        logits_to_keep: int | torch.Tensor = 0,
        **kwargs: Unpack[TransformersKwargs],
    ) -> CausalLMOutputWithPast:
        r"""
        示例：
        ```python
        >>> from transformers import AutoTokenizer, Qwen2ForCausalLM

        >>> model = Qwen2ForCausalLM.from_pretrained("meta-qwen2/Qwen2-2-7b-hf")
        >>> tokenizer = AutoTokenizer.from_pretrained("meta-qwen2/Qwen2-2-7b-hf")

        >>> prompt = "嘿，你有自主意识吗？能和我聊聊吗？"
        >>> inputs = tokenizer(prompt, return_tensors="pt")

        >>> # 生成文本
        >>> generate_ids = model.generate(inputs.input_ids, max_length=30)
        >>> tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        "嘿，你有自主意识吗？能和我聊聊吗？
        我没有自主意识，但我可以和你聊天。"
        ```"""
        outputs: BaseModelOutputWithPast = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            cache_position=cache_position,
            **kwargs,
        )

        hidden_states = outputs.last_hidden_state
        # 仅计算必要的对数几率（logits）；且若当前未在计算损失值，则无需将其向上转换（upcast）为浮点型
        slice_indices = slice(-logits_to_keep, None) if isinstance(logits_to_keep, int) else logits_to_keep
        logits = self.lm_head(hidden_states[:, slice_indices, :])

        loss = None
        if labels is not None:
            loss = self.loss_function(logits=logits, labels=labels, vocab_size=self.config.vocab_size, **kwargs)

        return CausalLMOutputWithPast(
            loss=loss,
            logits=logits,
            past_key_values=outputs.past_key_values,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [ ]:
# 定义 Qwen2ForSequenceClassification、Qwen2ForTokenClassification 和 Qwen2ForQuestionAnswering 类，分别继承自 GenericForSequenceClassification、GenericForTokenClassification 和 GenericForQuestionAnswering，以及 Qwen2PreTrainedModel
# 这些类负责实现 Qwen2 模型在序列分类、标记分类和问答任务中的功能，通常包括前向传播和损失计算等功能
class Qwen2ForSequenceClassification(GenericForSequenceClassification, Qwen2PreTrainedModel):
    pass

# 定义 Qwen2ForTokenClassification 类，继承自 GenericForTokenClassification 和 Qwen2PreTrainedModel，负责实现 Qwen2 模型在标记分类任务中的功能，通常包括前向传播和损失计算等功能
class Qwen2ForTokenClassification(GenericForTokenClassification, Qwen2PreTrainedModel):
    pass

# 定义 Qwen2ForQuestionAnswering 类，继承自 GenericForQuestionAnswering 和 Qwen2PreTrainedModel，负责实现 Qwen2 模型在问答任务中的功能，通常包括前向传播和损失计算等功能
class Qwen2ForQuestionAnswering(GenericForQuestionAnswering, Qwen2PreTrainedModel):
    base_model_prefix = "transformer"  # 该前缀名称与 Qwen2Model 中定义的前缀名称不同，确保在保存和加载模型时能够正确地处理问答模型的权重命名空间

In [ ]:
# 定义 `__all__` 列表，指定在使用 `from module import *` 语句时可以导入的类和函数的名称，以控制模块的公共接口并增强代码的可读性和可维护性
__all__ = [
    "Qwen2PreTrainedModel",
    "Qwen2Model",
    "Qwen2ForCausalLM",
    "Qwen2RMSNorm",
    "Qwen2ForSequenceClassification",
    "Qwen2ForTokenClassification",
    "Qwen2ForQuestionAnswering",
]